In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [8]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT article_id FROM candidate_articles WHERE content_type = 'ARTICLE'"
    cursor.execute(sql)
    candidates = cursor.fetchall()

In [2]:
import json

In [9]:
candidates = [x[0] for x in candidates]

In [11]:
len(candidates)

4522

In [10]:
candidates[0]

'65cb4bd88c9dc047f1e0407e'

In [16]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM user_article_interactions'
    cursor.execute(sql)
    interacted = cursor.fetchall()

In [17]:
interacted = [x[0] for x in interacted]

In [18]:
len(interacted)

16968

In [19]:
from sql.articles.MongoDBArticle import MongoDBArticle

In [20]:
combined = list(set(candidates).union(interacted))

In [21]:
len(combined)

20037

In [22]:
relevant_articles = MongoDBArticle.fetch_documents_by_ids(combined)

In [23]:
len(relevant_articles)

19975

In [27]:
article_full_contents = []

In [25]:
relevant_articles[0]

{'_id': ObjectId('651dc6e4a662d76276b7b7c3'),
 'url': 'https://economictimes.indiatimes.com/news/india/us-europe-and-india-need-to-create-competitive-alternative-model-to-chinas-coercive-production-model-rahul-gandhi/articleshow/103508725.cms',
 'source_id': ObjectId('6512cdcad01a9c8e86263e05'),
 '_class': 'com.wintWealth.insight.common.models.entities.Article',
 'image_url': 'https://economictimes.indiatimes.com/thumb/msid-103508762,width-2254,height-1280,imgsize-54818,overlay-economictimes/photo.jpg',
 'is_premium_article': False,
 'last_updated_time': '2023-09-08T17:27:00+05:30',
 'published_time': '2023-09-08T16:45:00+05:30',
 'short_description': "Congress leader Rahul Gandhi has called for the United States, Europe, and India to create an alternative and competitive model to China's coercive production model. Speaking at the Brussels Press Club, Gandhi emphasized the need for a democratic environment that offers political and economic freedom. ",
 'tags': ['Rahul Gandhi',
  'xi j

In [26]:
from src.data_models.Article import Article

In [28]:
for art in relevant_articles:
    if art['content_type'] == 'ARTICLE' and 'cleaned_text' in art:
        art_id = str(art['_id'])
        full_content = Article._generate_full_content(art['title'], art['cleaned_text'])
        article_full_contents.append({'article_id': art_id, 'full_content': full_content})

In [30]:
candidates_content_df = pd.DataFrame(article_full_contents)

In [31]:
candidates_content_df.to_csv('candidates_content_df.csv',index = False)

### Fetching responses from LLM

In [4]:
from src.llm_finetuning.constants import SYSTEM_PROMPT
from src.articles._utils import truncate_text_to_token_limit

In [5]:
candidates_content_df = pd.read_csv('candidates_content_df.csv')
category_classifier_test_df = pd.read_csv('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/notebooks/embeddings/titles_classes_gpt4_with_full_content.csv')
test = category_classifier_test_df[category_classifier_test_df.test_flag == 1]
test_articles_df = test[['id', 'full_content']].rename(columns = {'id': 'article_id'})
candidates_df = pd.concat([candidates_content_df, test_articles_df])

In [6]:
candidates_df

,article_id,full_content
0,651dc6e4a662d76276b7b7c3,"headline: US, Europe, and India need to create..."
1,651dc855a662d76276b7bbeb,headline: RBI issues revised norms for classif...
2,651dc87ba662d76276b7bc54,"headline: Gold, silver trade flat in line with..."
3,651dca46a662d76276b7c135,headline: Blue Dart rebrands delivery service ...
4,651dca5ca662d76276b7c16e,headline: Hunter Biden indicted in federal cas...
...,...,...
4962,65cb1397f4fc8477f2ed8f2a,title: Mazagon Dock Standalone December 2023 N...
4963,65c98a44f4fc8477f2ed7cb1,"title: Stock Market LIVE Updates | Nifty 50, S..."
4965,65cc2d0706df7536a8f6864b,title: Abbott's heart valve repair device gets...
4967,65c0878dc119ff6e57a3f042,title: Megatherm Induction share price makes a...


In [7]:
!pwd

/Users/ravi.tej/Desktop/ML/Recommendations/arcane/notebooks/llms/prompt-refinemenet-18-feb-2023


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B', trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def create_missing_article_details(article_id):
    r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': article_id})

In [138]:
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

In [213]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [214]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT article_id FROM llm_article_attributes WHERE model_finetune_id = 'OpenHermes_WatermelonUvulaMarigold'"
    cursor.execute(sql)
    results = cursor.fetchall()

In [215]:
completed = [x[0] for x in results]

In [216]:
len(completed)

32994

In [217]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT DISTINCT article_id FROM candidate_articles"
    cursor.execute(sql)
    results = cursor.fetchall()

In [218]:
union = [x[0] for x in results]

In [219]:
len(union)

3871

In [220]:
union = [x for x in union if x not in completed]

In [221]:
# union = list(candidates_df[~candidates_df.article_id.isin(completed)]['article_id'].unique())

In [222]:
import numpy as np

In [223]:
np.random.shuffle(union)

In [224]:
len(union)

1056

In [225]:
chunk_size = 12
num_chunks = len(union)//chunk_size + 1

In [ ]:
for i in range(num_chunks):
    article_chunk = union[i*chunk_size:(i+1)*chunk_size]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(create_missing_article_details, art_id) for art_id in article_chunk]
        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                completed_article_ids.append(response)
                continue
    print(f'completed chunk {(i + 1) * chunk_size} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Feb 24, 2024 18:32
completed chunk 12 articles at Feb 24, 2024 18:33 in 1.5166666666666666 minutes
starting creation at Feb 24, 2024 18:33
completed chunk 24 articles at Feb 24, 2024 18:35 in 1.4833333333333334 minutes
starting creation at Feb 24, 2024 18:35
completed chunk 36 articles at Feb 24, 2024 18:36 in 1.65 minutes
starting creation at Feb 24, 2024 18:36
completed chunk 48 articles at Feb 24, 2024 18:38 in 1.45 minutes
starting creation at Feb 24, 2024 18:38
completed chunk 60 articles at Feb 24, 2024 18:38 in 0.2 minutes
starting creation at Feb 24, 2024 18:38
completed chunk 72 articles at Feb 24, 2024 18:39 in 1.0 minutes
starting creation at Feb 24, 2024 18:39
completed chunk 84 articles at Feb 24, 2024 18:40 in 1.1833333333333333 minutes
starting creation at Feb 24, 2024 18:40
completed chunk 96 articles at Feb 24, 2024 18:42 in 1.75 minutes
starting creation at Feb 24, 2024 18:42
completed chunk 108 articles at Feb 24, 2024 18:44 in 1.7333333333333334

In [ ]:
2

In [42]:
5

5

In [43]:
2

2

In [44]:
3

3

In [45]:
4

4

In [10]:
# from botocore.config import Config
# import boto3

# my_config = Config(
#         region_name='ap-south-1',
#         connect_timeout=120,
#         read_timeout=120,
#         retries={
#             'max_attempts': 1,
#             'mode': 'standard'
#         },
#         max_pool_connections=40  # Increase the pool size
#     )

#     # Create a SageMaker Runtime client with the custom configuration
# sess1 = boto3.session.Session()
# sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)

In [9]:
# candidates_df['truncated_content'] = candidates_df['full_content'].apply(lambda x: truncate_text_to_token_limit(text=x, 
#                                                                                                                 encoder=tokenizer,
#                                                                                                                 token_limit=1536))

In [44]:
num_sys_prompt = 2560 - len(tokenizer.encode(SYSTEM_PROMPT))

In [108]:
def generate_prompt_for_row(row):
    truncated = truncate_text_to_token_limit(text=row['full_content'],encoder=tokenizer,token_limit=2560-num_sys_prompt-10)
    messages = [{"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"{truncated}\n"}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    return context_prompt

In [109]:
def generate_params_for_row(row):
    context_prompt = generate_prompt_for_row(row)
    parameters = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.5,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.02,
    "stop": ["###", "</s>", '<|im_end|>']
    }
    params = {"inputs": context_prompt, "parameters": parameters, "stream": False}
    params['parameters']['adapter_id'] = 'lorax/OpenHermes-2.5-Adapter-Attributes-WatermelonUvulaMarigold'
    params['parameters']['adapter_source'] = 's3'
    return params

In [110]:
import json

In [129]:
def get_attributes_for_candidate(row):
    my_config = Config(
        region_name='ap-south-1',
        retries={
            'max_attempts': 1,
            'mode': 'standard'
        },
        max_pool_connections=40  # Increase the pool size
    )
    sess1 = boto3.session.Session()
    sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)
    params = generate_params_for_row(row)
    response = sagemaker_runtime.invoke_endpoint(
            EndpointName='LORAX-Openhermes',
            Body=json.dumps(params),
            ContentType="application/json")
    r = response['Body'].read()
    print(f"response received for {row['article_id']}")
    try:
        attributes = json.loads(json.loads(r)[0]['generated_text'])
        if isinstance(attributes, dict):
            return row['article_id'], attributes, True
        else:
            try:
                attributes = json.loads(attributes)
                return row['article_id'], attributes, True
            except:
                return row['article_id'], attributes, False
    except:
        return row['article_id'], json.loads(r)[0]['generated_text'], False

In [101]:
params = generate_params_for_row(candidates_dict[20])

In [115]:
response = sagemaker_runtime.invoke_endpoint(
            EndpointName='LORAX-Openhermes',
            Body=json.dumps(params),
            ContentType="application/json")
r = response['Body'].read()

In [117]:
s = time.time()
a, b, c = get_attributes_for_candidate(candidates_dict[20])
print(time.time()-s)

45.74726986885071


In [67]:
import time
from datetime import datetime
from concurrent.futures import as_completed, ThreadPoolExecutor

In [118]:
def save_attributes(response_json):
    cur_df = pd.DataFrame(response_json, index=['i', ])
    cur_df.to_csv('candidates_lorax_responses.csv', header=None, mode='a')

In [131]:
responses_df = pd.read_csv('candidates_lorax_responses.csv',header=None)

In [132]:
candidates_dict = candidates_df[~candidates_df.article_id.isin(list(responses_df[1].unique()))].to_dict('records')

In [133]:
len(candidates_df)

20561

In [134]:
len(candidates_dict)

20446

In [135]:
valid_responses = []
invalid_responses = []

In [136]:
import json

In [137]:
start_time = time.time()
print(f'starting at {datetime.now()}')
valid_response_count = 0
invalid_response_count = 0
timedout = 0
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(get_attributes_for_candidate, article) for article in candidates_dict]
    for future in as_completed(futures):
        try:
            article_id, attributes, valid = future.result(timeout=60)
            if valid:  # Timeout or other errors
                valid_response_count += 1
                response_json = {'article_id': article_id, 'attributes': json.dumps(attributes), 'valid': True}
                valid_responses.append(response_json)
                if valid_response_count % 100 == 0:
                    print(f'completed {valid_response_count} in {time.time() - start_time} seconds')
            else:
                invalid_response_count += 1
                response_json = {'article_id': article_id, 'attributes': json.dumps(attributes), 'valid': False}
                invalid_responses.append(response_json)
                if invalid_response_count % 50 == 0:
                    print(f'now {invalid_response_count} invalid responses')
            save_attributes(response_json)
        except concurrent.futures.TimeoutError:
            timedout += 1
print(f'done in {time.time() - start_time} seconds')

starting at 2024-02-20 12:03:28.534511
response received for 652fc61d1e5cc42b1b139ed5
response received for 652fcacd1e5cc42b1b13a11f
response received for 652fcb091e5cc42b1b13a13e
response received for 652fcbbb1e5cc42b1b13a19a
response received for 652fcbdc1e5cc42b1b13a1ac
response received for 652fcfd41e5cc42b1b13a36b
response received for 652fcffe1e5cc42b1b13a380
response received for 652fcad81e5cc42b1b13a125response received for 652fc7f51e5cc42b1b139fb6

response received for 652fcbf51e5cc42b1b13a1b9
response received for 652fd00c1e5cc42b1b13a387
response received for 652fd01a1e5cc42b1b13a38d
response received for 652fd0481e5cc42b1b13a3a3
response received for 652fd13b1e5cc42b1b13a41b
response received for 652fd0561e5cc42b1b13a3aa
response received for 652fd23b1e5cc42b1b13a49b
response received for 652fd35d1e5cc42b1b13a529
response received for 652fd0521e5cc42b1b13a3a8
response received for 6530d40c1e5cc42b1b13a6cf
response received for 652fd1d21e5cc42b1b13a466
response received for


KeyboardInterrupt



response received for 653169101e5cc42b1b13cd15
response received for 653169111e5cc42b1b13cd2f
response received for 653169121e5cc42b1b13cdbb
response received for 653169121e5cc42b1b13cdb2response received for 653169101e5cc42b1b13cd0d
response received for 653169181e5cc42b1b13cf4c
response received for 653169181e5cc42b1b13cf73
response received for 653169181e5cc42b1b13cf9c
response received for 6531691a1e5cc42b1b13cfed
response received for 653169191e5cc42b1b13cfe1
response received for 6531691c1e5cc42b1b13d09a
response received for 653169241e5cc42b1b13d2fc
response received for 6531692b1e5cc42b1b13d48a
response received for 653169361e5cc42b1b13d668
response received for 65316a681e5cc42b1b13dc28
response received for 65316a971e5cc42b1b13dd23
response received for 6531694a1e5cc42b1b13d8b0
response received for 65316b631e5cc42b1b13dea7
response received for 65316a411e5cc42b1b13dbb7
response received for 653169601e5cc42b1b13d9ae
response received for 65316a721e5cc42b1b13dc54
response recei